In [10]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import datetime as dt

## twitter automatic login
- 환경: Windows 10
- 목표 환경: Ubuntu (WSL)
- 크롤링 코드 참고: https://jhnoru.tistory.com/59

In [13]:
"""
driver 가동
: web browser는 chrome 사용
"""
def init_driver():

    options = webdriver.ChromeOptions()
    #options.add_argument("--headless") 
    options.add_argument("--no-sandbox") 
    options.add_argument("--disable-gpu") 
    options.add_argument("window-size=1920x1080") 
    options.add_argument("lang=ko_KR")
    options.add_argument("user-agent=Chrome/85.0.4183.121")
    
    driver = webdriver.Chrome(options=options)
    driver.wait = WebDriverWait(driver, 5)
    
    return driver


def close_driver(driver):
    
    return

In [14]:
"""
크롬을 조작해서 트위터에 로그인
: 개인 id 및 pw이용
"""
def twitter_login(driver, username, password):
    driver.get("https://twitter.com/login")
    driver.implicitly_wait(3)
    driver.maximize_window()
    
    username_field = driver.find_element_by_name("session[username_or_email]")
    password_field = driver.find_element_by_name("session[password]")

    username_field.send_keys(username)
    driver.implicitly_wait(1)
    password_field.send_keys(password)
    driver.implicitly_wait(1)

    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/form/div/div[3]/div/div/span/span').submit()


자동 로그인까지 성공.   
검색어 query로 크롤링 시도하고자함.

### 트위터 크롤링 연습

In [15]:
def crawling(driver, start_year, start_month, start_day, until_year, until_month, until_day):
    startdate = dt.date(year = start_year,month = start_month, day = start_day)
    untildate = dt.date(year = until_year,month = until_month, day = until_day)
    
    # 검색어: 경제(추후 수정 필요)
    TWITTER_URL = "https://twitter.com/search?q=%EA%B2%BD%EC%A0%9C%20until%3A" + str(untildate) + "%20since%3A"+ str(startdate) +"%20-filter%3Alinks&src=typed_query"

    driver.maximize_window()
    driver.get(TWITTER_URL)
    
    # 최신 탭 클릭
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div[1]/div[2]/nav/div/div[2]/div/div[2]/a').click()

    print('The scroll is starting to move bottom')

    # 페이지 스크롤을 끝날 때까지 계속 내림
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            # Wait to load page
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if(new_height == last_height):
                break

        last_height = new_height
    print('Arrived at the end of the page')
    print('Start twitter crawling')
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 날짜 저장된 태그: day
    day = soup.find_all("a", {"class": "r-1re7ezh r-1loqt21 r-1q142lx r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-3s2u2q r-qvutc0 css-4rbku5 css-18t94o4 css-901oao"})
    # 트윗 글 저장된 태그: tweets
    tweets = soup.find_all("div", {"class": "css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0"})
    return day, tweets

In [18]:
driver = init_driver()
twitter_login(driver, 'twitter_user_id', 'twitter_user_password')

day, tweets = crawling(driver, 2020, 9, 29, 2020, 9, 30)
driver.quit()

The scroll is starting to move bottom
Arrived at the end of the page
Start twitter crawling


ISSUE    
- 기술적 이슈: 전체 글이 긁어와지지않음. 더 분석해봐야할듯!    
스크롤을 다 내려도, 해당 태그에 대한 검색이 한번에 되질 않음. 
- 키워드 조정

In [20]:
# 날짜 저장된 태그
for d in day:
    print(d.get_text())

9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일
9월 29일


In [22]:
# 날짜 저장된 태그
for t in tweets:
    print(t.get_text())
    print()

기업인과 유명 인사 등 고소득층이 미국 50개 주 가운데 경제 규모 1위인 캘리포니아를 떠나고 있음. 주의회가 최근 최고 세율 인상을 추진하고 있는 데다 다른 주에 비해 높은 법인세와 촘촘한 규제에 고소득층이 등을 돌린 것임. 캘리포니아주 소득세 최고세율은 13.3%에 달함. #매세지200929

한국정치 사회 경제
재미없고 메이저리그
유현진 김광현 최지만
모두 퍼스트시즌 진출
했는데 유현진 최 지만 선후배가 맞붙는 다네
요 운명의 작난인가 누굴응원해 김광현팀 은 퍼스트진출했다면
우승가능성이 많고 유럽축구 손홍민은 하 늘높은줄 모르고 날고 고 황희찬이도 파닥거 리고있다

아메리카노 4잔 
카페라떼 4잔(이미마심)
몬스터1캔으로 
어젯저녁 다섯시부터 오늘 밤 12시까지 버틸계획인데 아무래도 밤샘 경제적 손실이 너무 큼

ⓑ 계획경제체제(정부명령에의한 경제운용)
ⓑ-2. 생산수단의 국유화, 정부관료에의한 결정, 통제에 의한 자원배분
한계: 경제적 유인의 부족으로 창의성과 활력 낮음, 잘못된 계획경제로 경제발전 저해

국민들 스스로 본인들 안전 지키는걸 가장 먼저 내세워야하는것을 ..
정부가 시진핑 방한 노리고 중국 눈치보고 경제와 국민안전 저울질 하던걸 나서서 쉴드치던 꼴이 뭐랄까....

기억났다 그냥 개뻘소리로 경제때 님 반인줄 알고 낙서한 거 몰래 껴놓으려고 님 책상 찾았다는 말이었음

[속보] 코로나19 어제 확진자 38명…국내 발생 23명

다들 고셉 추리하고 있을때 난 상업경제 공부를 하지...

저는 진짜 얀님이 제가 트위터를 접게되는 미지의 그날 전에 꼭 경제적 자유를 쟁취하게 되길 바랍니다. 그럴 자격이 있어요. 응원합니다. 우리모두 다같이 잘먹고 잘살아보세!

ㅇㄴ 뭔가 경제 전문이실 것 같은데요ㅋㅋㅋㅋㅋㅋㅋㅋ

또한 신자유주의와 경제 위기의 가장 큰 희생자인 청년 세대, 무엇보다도 금융수탈 체제와 불안정 노동사회의 문제점에 대해 이미 자각했으나 정치적 집결지를 아직 찾지 못한 1000만 비정규불안정노동자들이야말로 좌파당의 진정한 출발점이다.

